In [32]:
from multiprocessing import Pool

def create_lag(df_code,n_lag=[3,7,14,],shift_size=1):
    code = df_code['assetCode'].unique()
    
    for col in return_features:
        for window in n_lag:
            rolled = df_code[col].shift(shift_size).rolling(window=window)
            lag_mean = rolled.mean()
            lag_max = rolled.max()
            lag_min = rolled.min()
            lag_std = rolled.std()
            df_code['%s_lag_%s_mean'%(col,window)] = lag_mean
            df_code['%s_lag_%s_max'%(col,window)] = lag_max
            df_code['%s_lag_%s_min'%(col,window)] = lag_min
#             df_code['%s_lag_%s_std'%(col,window)] = lag_std
    return df_code.fillna(-1)

def generate_lag_features(df,n_lag = [3,7,14]):
    features = ['time', 'assetCode', 'assetName', 'volume', 'close', 'open',
       'returnsClosePrevRaw1', 'returnsOpenPrevRaw1',
       'returnsClosePrevMktres1', 'returnsOpenPrevMktres1',
       'returnsClosePrevRaw10', 'returnsOpenPrevRaw10',
       'returnsClosePrevMktres10', 'returnsOpenPrevMktres10',
       'returnsOpenNextMktres10', 'universe']
    
    assetCodes = df['assetCode'].unique()
    print(assetCodes)
    all_df = []
    df_codes = df.groupby('assetCode')
    df_codes = [df_code[1][['time','assetCode']+return_features] for df_code in df_codes]
    print('total %s df'%len(df_codes))
    
    pool = Pool(4)
    all_df = pool.map(create_lag, df_codes)
    
    new_df = pd.concat(all_df)  
    new_df.drop(return_features,axis=1,inplace=True)
    pool.close()
    
    return new_df

def mis_impute(data):
    for i in data.columns:
        if data[i].dtype == "object":
            data[i] = data[i].fillna("other")
        elif (data[i].dtype == "int64" or data[i].dtype == "float64"):
            data[i] = data[i].fillna(data[i].mean())
        else:
            pass
    return data

def data_prep(market_train):
    lbl = {k: v for v, k in enumerate(market_train['assetCode'].unique())}
    market_train['assetCodeT'] = market_train['assetCode'].map(lbl)
    market_train = market_train.dropna(axis=0)
    return market_train

fcol=['volume', 'close', 'open', 'returnsClosePrevRaw1', 'returnsOpenPrevRaw1', 'returnsClosePrevMktres1', 'returnsOpenPrevMktres1', 'returnsClosePrevRaw10', 'returnsOpenPrevRaw10', 'returnsClosePrevMktres10', 'returnsOpenPrevMktres10', 'returnsClosePrevMktres10_lag_3_mean', 'returnsClosePrevMktres10_lag_3_max', 'returnsClosePrevMktres10_lag_3_min', 'returnsClosePrevMktres10_lag_7_mean', 'returnsClosePrevMktres10_lag_7_max', 'returnsClosePrevMktres10_lag_7_min', 'returnsClosePrevMktres10_lag_14_mean', 'returnsClosePrevMktres10_lag_14_max', 'returnsClosePrevMktres10_lag_14_min', 'returnsClosePrevRaw10_lag_3_mean', 'returnsClosePrevRaw10_lag_3_max', 'returnsClosePrevRaw10_lag_3_min', 'returnsClosePrevRaw10_lag_7_mean', 'returnsClosePrevRaw10_lag_7_max', 'returnsClosePrevRaw10_lag_7_min', 'returnsClosePrevRaw10_lag_14_mean', 'returnsClosePrevRaw10_lag_14_max', 'returnsClosePrevRaw10_lag_14_min', 'open_lag_3_mean', 'open_lag_3_max', 'open_lag_3_min', 'open_lag_7_mean', 'open_lag_7_max', 'open_lag_7_min', 'open_lag_14_mean', 'open_lag_14_max', 'open_lag_14_min', 'close_lag_3_mean', 'close_lag_3_max', 'close_lag_3_min', 'close_lag_7_mean', 'close_lag_7_max', 'close_lag_7_min', 'close_lag_14_mean', 'close_lag_14_max', 'close_lag_14_min', 'assetCodeT']

In [2]:
import pandas as pd
import numpy as np

In [25]:
market_test_df = pd.read_csv("../../data/market_test_df.csv").drop('Unnamed: 0', axis=1)
market_test_df['time'] = pd.to_datetime(market_test_df['time'])

In [26]:
dates = market_test_df['time'].unique()
market_test_df[market_test_df['time'] == dates[0]].tail()

,time,assetCode,assetName,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevMktres1,returnsOpenPrevMktres1,returnsClosePrevRaw10,returnsOpenPrevRaw10,returnsClosePrevMktres10,returnsOpenPrevMktres10,returnsOpenNextMktres10,universe
875,2016-10-13 22:00:00,ZION.O,Zions Bancorp,3041647.0,30.36,30.78,-0.027235,-0.017241,-0.023449,-0.014122,-0.005242,-0.004528,0.004824,0.015436,0.060453,1.0
876,2016-10-13 22:00:00,ZIOP.O,ZIOPHARM Oncology Inc,1042606.0,5.22,5.19,-0.003817,-0.038889,0.007138,-0.026353,-0.064516,-0.115843,-0.060812,-0.105750,0.034282,0.0
877,2016-10-13 22:00:00,ZLTQ.O,ZELTIQ Aesthetics Inc,358309.0,40.83,40.00,0.012649,-0.003736,0.017270,-0.001255,0.064113,0.013942,0.059521,0.005566,-0.189523,0.0
878,2016-10-13 22:00:00,ZNGA.O,Zynga Inc,13474904.0,2.90,2.91,-0.013605,0.003448,-0.008828,0.006002,0.032028,0.024648,0.022769,0.002984,-0.039757,1.0
879,2016-10-13 22:00:00,ZTS.N,Zoetis Inc,2812177.0,51.02,51.13,-0.007779,0.001960,-0.003743,0.006036,0.000000,-0.011790,0.011654,0.008790,-0.046295,1.0


In [27]:
import pickle as pk
_ref_models = pk.load(open("../pickle/_ref_models.pkl","rb"))

In [28]:
# load models
gbm_1 = _ref_models[0]
gbm_2 = _ref_models[1]
gbm_3 = _ref_models[2]
gbm_4 = _ref_models[3]
gbm_5 = _ref_models[4]
gbm_6 = _ref_models[5]

In [34]:
mins,maxs,rng = pk.load(open("../pickle/normalizing.pkl","rb"))

In [ ]:
import time

PREDICTIONS = []

#artificial prediction
#days = env.get_prediction_days()
days = []
for date in dates:
    market_obs_df = market_test_df[market_test_df['time'] == date]
    predictions_template_df = pd.DataFrame({'assetCode':market_test_df[market_test_df['time'] == date]['assetCode'],
                                            'confidenceValue':0.0})
    days.append([market_obs_df,None,predictions_template_df])
 

n_days = 0
n_lag=[3,7,14]
prep_time = 0
prediction_time = 0
packaging_time = 0
total_market_obs_df = []
for (market_obs_df, news_obs_df, predictions_template_df) in days:
    n_days +=1
    if (n_days%50==0):
        pass
        #print(n_days,end=' ')
    t = time.time()
    market_obs_df['time'] = market_obs_df['time'].dt.date
    
    return_features = ['returnsClosePrevMktres10','returnsClosePrevRaw10','open','close']
    total_market_obs_df.append(market_obs_df)
    if len(total_market_obs_df)==1:
        history_df = total_market_obs_df[0]
    else:
        history_df = pd.concat(total_market_obs_df[-(np.max(n_lag)+1):])
    new_df = generate_lag_features(history_df,n_lag=[3,7,14])
    market_obs_df = pd.merge(market_obs_df,new_df,how='left',on=['time','assetCode'])
    market_obs_df = mis_impute(market_obs_df)
    market_obs_df = data_prep(market_obs_df)
    
    X_live = market_obs_df[fcol].values
    X_live = 1 - ((maxs - X_live) / rng)
    prep_time += time.time() - t
    t = time.time()
    lp = (gbm_1.predict(X_live) + gbm_2.predict(X_live)+gbm_3.predict(X_live)+gbm_4.predict(X_live)+gbm_5.predict(X_live)+gbm_6.predict(X_live))/6
    prediction_time += time.time() -t
    t = time.time()
    confidence = lp
    confidence = (confidence-confidence.min())/(confidence.max()-confidence.min())
    #print("max_min_predict",max_min_predict)
    confidence = confidence * 2 - 1
    preds = pd.DataFrame({'assetCode':market_obs_df['assetCode'],'confidence':confidence})
    predictions_template_df = predictions_template_df.merge(preds,how='left').drop('confidenceValue',axis=1).fillna(0).rename(columns={'confidence':'confidenceValue'})
    PREDICTIONS.append(predictions_template_df)
    packaging_time += time.time() - t
    
#env.write_submission_file()
#sub  = pd.read_csv("submission_versionnew.csv")

/Users/alex/Desktop/Coding/AI/two-sigma-kaggle/env3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


['LGIH.O' 'LGND.O' 'LH.N' 'LHO.N' 'LII.N' 'LILA.O' 'LILAK.O' 'LITE.O'
 'LIVN.O' 'LKQ.O' 'LLTC.O' 'LLY.N' 'LM.N' 'LMCK.O' 'LMNX.O' 'LMT.N' 'LN.N'
 'LNC.N' 'LNCE.O' 'LNG.A' 'LNKD.N' 'LNT.N' 'LOCK.N' 'LOGI.O' 'LOGM.O'
 'LOPE.O' 'LOW.N' 'LPI.N' 'LPL.N' 'LPLA.O' 'LPNT.O' 'LPT.N' 'LPX.N' 'LQ.N'
 'LRCX.O' 'LSCC.O' 'LSI.N' 'LSTR.O' 'LSXMA.O' 'LSXMK.O' 'LTRPA.O' 'LTXB.O'
 'LUK.N' 'LULU.O' 'LUV.N' 'LVLT.N' 'LVS.N' 'LXK.N' 'LXP.N' 'LXRX.O'
 'LYB.N' 'LYG.N' 'LYV.N' 'LZB.N' 'M.N' 'MA.N' 'MAA.N' 'MAC.N' 'MAN.N'
 'MANH.O' 'MAR.O' 'MAS.N' 'MASI.O' 'MAT.O' 'MBFI.O' 'MBI.N' 'MBT.N'
 'MCD.N' 'MCHP.O' 'MCK.N' 'MCO.N' 'MD.N' 'MDC.N' 'MDCO.O' 'MDLZ.O' 'MDP.N'
 'MDRX.O' 'MDSO.O' 'MDT.N' 'MDU.N' 'MDXG.O' 'MEG.N' 'MELI.O' 'MENT.O'
 'MEOH.O' 'MET.N' 'MFA.N' 'MFC.N' 'MFS.N' 'MGA.N' 'MGLN.O' 'MGM.N'
 'MGNX.O' 'MGP.N' 'MHK.N' 'MHLD.O' 'MIC.N' 'MIDD.O' 'MIK.O' 'MINI.O'
 'MITL.O' 'MJN.N' 'MKC.N' 'MKSI.O' 'MKTX.O' 'MLHR.O' 'MLM.N' 'MLNX.O'
 'MMC.N' 'MMM.N' 'MMP.N' 'MMS.N' 'MMSI.O' 'MNK.N' 'MNR.N' 'MNRO.O'
 'MNST.O' '

total 1878 df
['LGIH.O' 'LGND.O' 'LH.N' ... 'CORT.O' 'RRD.N' 'WTW.N']
total 1883 df
['LGIH.O' 'LGND.O' 'LH.N' ... 'RRD.N' 'WTW.N' 'VC.N']
total 1884 df
['LGIH.O' 'LGND.O' 'LH.N' ... 'RRD.N' 'WTW.N' 'VC.N']
total 1884 df
['A.N' 'AAL.O' 'AAN.N' ... 'WTW.N' 'VC.N' 'QUAD.N']
total 1879 df
['A.N' 'AAL.O' 'AAN.N' ... 'TEP.N' 'VEDL.N' 'WEX.N']
total 1883 df


In [ ]:
PRETRAINED = True
if PRETRAINED:
    import sys;sys.path.append('../')
    import os
    os.listdir('../pickle')
    model = model('DecisionTree.model_lgbm_71')
    model._load()

In [ ]:
#prediction
#days = env.get_prediction_days()
n_days = 0
prep_time = 0
prediction_time = 0
packaging_time = 0
total_market_obs_df = []
for (market_obs_df, news_obs_df, predictions_template_df) in days:
    n_days +=1
    if (n_days%50==0):
        pass
        #print(n_days,end=' ')
    t = time.time()
    market_obs_df['time'] = market_obs_df['time'].dt.date

    total_market_obs_df.append(market_obs_df)
    if len(total_market_obs_df)==1:
        history_df = total_market_obs_df[0]
    else:
        history_df = pd.concat(total_market_obs_df[-(np.max(n_lag)+1):])
        
    confidence = model.predict([history_df, None],verbose=True,normalize=[maxs,mins])      
        
    preds = pd.DataFrame({'assetCode':market_obs_df['assetCode'],'confidence':confidence})
    predictions_template_df = predictions_template_df.merge(preds,how='left').drop('confidenceValue',axis=1).fillna(0).rename(columns={'confidence':'confidenceValue'})
    env.predict(predictions_template_df)
    packaging_time += time.time() - t